In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
sns.set_context('paper', font_scale=1.5)

heatmap_cmap = 'GnBu'

import scipy
from scipy.stats import ttest_rel, beta, ttest_ind
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
import numpy as np
import os
import yaml
import shutil
from collections import OrderedDict

from utils import quick_load_experiments_tsv, get_significance_by_column, add_probing_location_better_column
from common import lang2code, code2lang, lang2family, family2langs, lang2large_family

figure_dir = "fig/typology"
os.makedirs("fig", exist_ok=True)
os.makedirs(figure_dir, exist_ok=True)

np.random.seed(2)

%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
areas = pd.read_csv("../dataframes/wals/wals_areas.tsv", sep="\t")
feat2area = dict(areas[['Name', 'Area']].drop_duplicates().values)
feat2area['Family'] = 'Family'

wals_languages = []
wals = []
param2area = {}
for fn in os.listdir('../dataframes/wals'):
    if not fn.endswith('csv'):
        continue
    lang = fn.split(".")[0]
    wals_languages.append(lang)
    df = pd.read_csv(f"../dataframes/wals/{lang}.csv", encoding='latin1')
    df = df.sort_values('Parameter').pivot(index='Language', columns='Parameter', values='Name')
    wals.append(df)
    
wals = pd.concat(wals, sort=True)
wals.index = wals_languages
wals.loc['Latin'] = np.nan 
wals['Family'] = wals.index.map(lambda l: lang2family[l])
print(wals.shape), wals.isnull().stack().mean()
wals

(39, 177)


<ipython-input-34-1ee5a8b68884>:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  wals['Family'] = wals.index.map(lambda l: lang2family[l])


Parameter,'Want' Complement Subjects,'When' Clauses,Absence of Common Consonants,Action Nominal Constructions,Adjectives without Nouns,Alignment of Case Marking of Full Noun Phrases,Alignment of Case Marking of Pronouns,Alignment of Verbal Person Marking,Antipassive Constructions,Applicative Constructions,...,Verbal Person Marking,Voicing and Gaps in Plosive Systems,Voicing in Plosives and Fricatives,Vowel Nasalization,Vowel Quality Inventories,Weight Factors in Weight-Sensitive Stress Systems,Weight-Sensitive Stress,Zero Copula for Predicate Nominals,Zero Marking of A and P Arguments,Family
Danish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Long vowel or coda consonant,Combined: Right-edge and unbounded,NaN,NaN,Germanic
Lithuanian,NaN,NaN,All present,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None missing in /p t k b d g/,In both plosives and fricatives,NaN,Average (5-6),Lexical stress,Unbounded: Stress can be anywhere,Possible,NaN,Baltic
Swedish,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Long vowel or coda consonant,Right-oriented: One of the last three,Impossible,NaN,Germanic
Catalan,NaN,NaN,All present,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None missing in /p t k b d g/,In both plosives and fricatives,NaN,Large (7-14),Lexical stress,Right-edge: Ultimate or penultimate,NaN,NaN,Romance
Italian,NaN,Balanced/deranked,NaN,NaN,NaN,NaN,NaN,Accusative,NaN,NaN,...,Only the A argument,NaN,NaN,NaN,NaN,Lexical stress,Right-edge: Ultimate or penultimate,NaN,NaN,Romance
Urdu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Indic
English,Subject is left implicit,Balanced/deranked,All present,Mixed,Marked by following word,Neutral,Nominative - accusative (standard),Accusative,No antipassive,No applicative construction,...,Only the A argument,None missing in /p t k b d g/,In both plosives and fricatives,Contrast absent,Large (7-14),Long vowel or coda consonant,Right-oriented: One of the last three,Impossible,Non-zero marking,Germanic
Dutch,NaN,NaN,NaN,Ergative-Possessive,NaN,NaN,NaN,Accusative,NaN,NaN,...,Only the A argument,NaN,NaN,NaN,NaN,Coda consonant,Right-oriented: One of the last three,Impossible,NaN,Germanic
Armenian,NaN,NaN,All present,Possessive-Accusative,NaN,Nominative - accusative (standard),Nominative - accusative (standard),Accusative,No antipassive,No applicative construction,...,Only the A argument,Other,In fricatives alone,Contrast absent,Average (5-6),Prominence,Unbounded: Stress can be anywhere,Impossible,NaN,Armenian
Afrikaans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Germanic


In [33]:
wals = pd.read_csv('../dataframes/wals.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../dataframes/wals.csv'

In [35]:
wals.notnull().sum().sum()  / (wals.shape[0]*wals.shape[1])

0.536723163841808

In [30]:
wals = wals.drop(columns=['Unnamed: 0'],axis=1)
wals.notnull().sum().idxmax()

'The Velar Nasal'

0.46327683615819204

In [32]:
wals['The Velar Nasal'].notnull().sum()
wals['The Velar Nasal']

0               No velar nasal
1               No velar nasal
2               No velar nasal
3               No velar nasal
4               No velar nasal
                 ...          
2657            No velar nasal
2658            No velar nasal
2659            No velar nasal
2660                       NaN
2661    No initial velar nasal
Name: The Velar Nasal, Length: 2662, dtype: object